In [43]:
import requests
import pandas as pd
from datetime import datetime

In [44]:
url = "https://unofficial-redfin.p.rapidapi.com/properties/list"

querystring = {
    "region_id": "4899",
    "region_type": "6",
    "uipt": "1,2,3,4,7,8",
    "status": "9",
    "sf": "1,2,3,5,6,7",
    "num_homes": "300"
}

headers = {
    "X-RapidAPI-Key": "b60faef7a6msh2c985c19232815ap1c4e46jsna04a608045c9",
    "X-RapidAPI-Host": "unofficial-redfin.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
data = response.json()

# Extracting nested data from 'homes' -> 'homeData'
homes_data = [home['homeData'] for home in data['homes']]


# Convert the extracted data to pandas DataFrame
df = pd.json_normalize(homes_data)

# Get the current date and time
current_datetime = datetime.now()

# Add the current date and time as a new column in the DataFrame
df['Data Requested Date'] = current_datetime



In [45]:
import re

# Function to format column names
def format_column_name(column_name):
    # Replace periods with spaces
    column_name = column_name.replace('.', ' ')

    # Add space before capital letters and make lowercase
    formatted_name = ''.join([' ' + char if char.isupper() else char for char in column_name]).strip().lower()
    
    # Remove leading/trailing spaces and replace multiple spaces with a single space
    formatted_name = re.sub(' +', ' ', formatted_name.strip()).lower()

    return formatted_name
# Format the column headers
df.columns = [format_column_name(col) for col in df.columns]

In [46]:
# Convert 'days on market days on market value' to numeric
df['days on market days on market value'] = pd.to_numeric(df['days on market days on market value'], errors='coerce')
# Convert 'days on market days on market value' to a timedelta and subtract from 'data requested date'
df['date added to market'] = df['data requested date'] - pd.to_timedelta(df['days on market days on market value'], unit='D')


In [47]:
df['days on market time on redfin seconds'] = pd.to_numeric(df['days on market time on redfin seconds'], errors='coerce')
# Convert 'days on market time on redfin seconds' to a timedelta and subtract from 'data requested date'
df['date added to redfin'] = df['data requested date'] - pd.to_timedelta(df['days on market time on redfin seconds'], unit='s')

In [48]:
# Convert 'last sale data last sold date seconds' to numeric, filling NaNs with 0
df['last sale data last sold date seconds'] = pd.to_numeric(df['last sale data last sold date seconds'], errors='coerce').fillna(0)

# Apply the calculation to all rows
df['date of last sale'] = df['data requested date'] - pd.to_timedelta(df['last sale data last sold date seconds'], unit='s')

# For rows where 'last sale data last sold date seconds' is 0, set 'date of last sale' to NaN or some default value
df.loc[df['last sale data last sold date seconds'] == 0, 'date of last sale'] = pd.NaT  # or some other default value


In [49]:
#check for duplicates
df_new = df
df_existing = pd.read_excel('data/dshs_test.xlsx')
current_date = datetime.now()


In [50]:
# Ensure the columns are of the same data type
df_existing['listing id value'] = df_existing['listing id value'].astype(str)
df_existing['property id'] = df_existing['property id'].astype(str)
df_new['listing id value'] = df_new['listing id value'].astype(str)
df_new['property id'] = df_new['property id'].astype(str)

# Identify duplicates in the new data based on 'listing id value' and 'property id'
duplicates = df_new[['listing id value', 'property id']].isin(df_existing[['listing id value', 'property id']].to_dict(orient='list')).all(axis=1)

# Filter out duplicates and append the non-duplicates to the existing data
df_existing = df_existing.append(df_new[~duplicates], ignore_index=True)

# Identify homes that are in the existing data but not in the new data
sold_homes = ~df_existing[['listing id value', 'property id']].isin(df_new[['listing id value', 'property id']].to_dict(orient='list')).all(axis=1)

# Update 'sold' status and 'date of last sale' for these homes
df_existing.loc[sold_homes, 'sold'] = True
df_existing.loc[sold_homes, 'date of last sale'] = current_date

/var/folders/tw/nqkxq7jn1msgjm8k_0dkcwlw0000gn/T/ipykernel_19669/333402078.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_existing = df_existing.append(df_new[~duplicates], ignore_index=True)


In [51]:
df_existing.to_excel('data/dshs_test.xlsx')